## Monash Handbook and Prequisites Documentation Project




### Determining prerequisites

I've written a script to scrape prerequisites from `https://mscv.apps.monash.edu` which validates a course map. Since it gives us prerequisites if we're missing them, we can simply submit up to 125 of those requests in a go. That being said, 
we still need a way to determine corequisites and prohibitions:
- As far as prohibitions are concerned, we can scrape from the handbook, then do a simple [A-Z]^3[0-9]^4 regex check on the prohibitions section. The API will return all the prohibited units.
- Corequisites are more tricky, however we should be able to make simplifications to our process. For instance, we can regex check the coreq section and then not include them on second pass for units that have a corequisite.

We now explore the data source. It takes 13.29 seconds on my PC to run this:

In [6]:
!python src/get_all_prereqs.py

13.293525218963623


Yields a file named `unit_reqs_clean.json`. I've preprocessed and removed a fair bit of the data the server sends as it's meaningless, however we still need to inspect the data.

In [7]:
import json

In [8]:
with open("unit_reqs_clean.json", "r") as file:
    unit_reqs = json.load(file)

For any given unit, it can have a list of prerequisites, corequisites, prohibitions and any other requirements. There are 8 different messages that you may get for unit enrollment:

In [33]:
unique_messages = set([item['title'] for sublist in unit_reqs.values() for item in sublist])
for msg in unique_messages:
    print(msg)

Prohibited unit
Have not enrolled in a unit
Have not completed enough units
Have not passed enough units
Not enough passed credit points
Not enough enrolled credit points
Missing corequisites
Permission is required for this unit


So let's go through them:

- Prohibited unit: You've enrolled/completed in a unit that prevents you from taking the current unit. For example, MTH1030 and ENG1005 both prohibit each other. A thing to note is that you will be given the names of all the prohibited units with that, e.g enrolling in ENG1005 and MTH1030 will also tell you MTH1035 is prohibited

- Have not enrolled in a unit: This one is unusual, as it only appears for 12 units. It may say to enrol in a list of units, however it really means to have done it as a prerequisite. EAE2522 is one such example

- Have not completed enough units: Again, this only appears for 3 units, all of which have the prefix APG. This seems to just be a completion requirement.

- Have not passed enough units: This is the normal message if you lack the prerequisites for a unit. Appears in most places.

- Not enough passed credit points: Some units simply require `x` credit points before you can enrol in them. Some mandate `y` credit points from faculty `z`. This appears less often but there are 360 occurences.

- Not enough enrolled credit points: Only appears once, but seems to be similar to the above, EDF5019

- Missing corequisites: Corequisites are a special sort of prerequisite that can be taken either before you do a unit, or concurrently with the unit. For instance, ENG1014 has a corequisite for ENG1005.

- Permission is required for this unit: You need to contact someone in order to enrol in this unit. Fairly standard.

In [10]:
unit_reqs["FIT3047"]

[{'title': 'Not enough passed credit points',
  'description': 'You must pass 90 more credit points before you can enrol into this unit.'},
 {'title': 'Have not passed enough units',
  'description': 'Please enrol in 2 of these units: FIT1001 or FIT1005'},
 {'title': 'Have not passed enough units',
  'description': 'Please enrol in 1 of these units: FIT1040'},
 {'title': 'Have not passed enough units',
  'description': 'Please enrol in 1 of these units: FIT1008, FIT1035, FIT2034, FIT2071, FIT2076, FIT2081 or FIT2104'},
 {'title': 'Have not passed enough units',
  'description': 'Please enrol in 1 of these units: FIT1004 or FIT2094'},
 {'title': 'Have not passed enough units',
  'description': 'Please enrol in 2 of these units: FIT2001 or FIT2002'},
 {'title': 'Have not passed enough units',
  'description': 'Please enrol in 1 of these units: FIT1003, FIT1049 or FIT2003'}]